# [MOTrainer: Measurement Operator Trainer](https://zenodo.org/record/7540443)

An example notebook that split data to train-test and and uses in DNN trainings.

## Import libraries

In [1]:
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt # only for plots
import pickle
import pandas as pd
import logging
import sys
import motrainer
import dask_ml.model_selection as dcv
from motrainer.jackknife import JackknifeGPI

2023-12-05 14:28:38.199970: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


data saved from the notebook [1_example_one_pickle_file_with_nested_fields](./1_example_one_pickle_file_with_nested_fields.ipynb)

## Read data and split to train and test datasets

In [2]:
# Read the data
zarr_file_path = "./example1_data.zarr"
ds = xr.open_zarr(zarr_file_path)

In [3]:
def to_dataframe(ds):
    return ds.to_dask_dataframe()

def chunk(ds, chunks):
    return ds.chunk(chunks)
    
bags = motrainer.dataset_split(ds, "space")
bags = bags.map(chunk, {"space": 100}).map(to_dataframe)

test_size = 0.33
f_shuffle = True
train_test_bags = bags.map(
    dcv.train_test_split, test_size=test_size, shuffle=f_shuffle, random_state=1
)  
train_bags = train_test_bags.pluck(0)
test_bags = train_test_bags.pluck(1)

## Define training parameters

In [4]:
# path to store results
out_path = './dnn_examples/results'

# JackKnife parameters
val_split_year = 2017
output_list = ['sig', 'slop', 'curv']
input_list = ['TG1', 'TG2', 'TG3', 'WG1', 'WG2', 'WG3', 'BIOMA1', 'BIOMA2']

# Training parameters
searching_space = {
    'num_dense_layers': [1, 10],
    'num_input_nodes': [1, 6],
    'num_dense_nodes': [1, 128],
    'learning_rate': [5e-4, 1e-2],
    'activation': ['relu']
}

# Here, I reduce parameters to be able to run on my own machine
optimize_space = {
    'best_loss': 2, # 1
    'n_calls': 11, # 15
    'epochs': 5, # 300
    'noise': 0.1, 
    'kappa': 5,
    'validation_split': 0.2,
    'x0': [1e-3, 1, 4, 13, 'relu', 64]
} # For weightling loss: 'loss_weights': [1, 1, 0.5], 

## Run the training

In this example, we will demonstrate how to run the training parralel pergrid with a dask cluster.

To add a Dask cluster to this notebook, you can use the Dask JupyterLab extension (look for the Dask logo on the left tab of the JupyterLab interface):

Click on the Dask logo;
click the `Scale` button, set up the number of workers to the number of available cores;
then click `<>` to add a code block.
Then a code cell will be added to this notebook. Please drop this cell below. By executing it, a Dask SLURMCluster will be created.

--ADD DASK SLURMCluster HERE--

This is an example code of DASK cluster

```python
from dask.distributed import Client

client = Client("tcp://127.0.0.1:39087")
client
```

In [6]:
aprior, post = [], []
# Loop all gpi
for gpi_num, df in enumerate(train_bags):
    
    # remove NA data
    gpi_data = df.compute()
    gpi_data.dropna(inplace=True)

    # add time to index
    gpi_data.set_index("time", inplace=True, drop=True)

    gpi = JackknifeGPI(gpi_data,
                       val_split_year,
                       input_list,
                       output_list,
                       outpath=f'{out_path}/gpi{gpi_num+1}')

    gpi.train(searching_space=searching_space,
              optimize_space=optimize_space,
              normalize_method='standard',
              training_method='dnn',
              performance_method='rmse',
              verbose=2)

    gpi.export_best()

    # Export apriori performance
    path_apriori_performance = f'{gpi.outpath}/apriori_performance_{gpi.best_year}'
    with open(path_apriori_performance, 'wb') as f:
        pickle.dump(gpi.apr_perf, f)

    # Export postpriori performance
    path_postpriori_performance = f'{gpi.outpath}/postpriori_performance_{gpi.best_year}'
    with open(path_postpriori_performance, 'wb') as f:
        pickle.dump(gpi.post_perf, f)

    aprior.append(gpi.apr_perf)
    post.append(gpi.post_perf)
    print("=========================================")
    print(f"       GPI {gpi_num + 1} done")
    print("       aprior performance(RMSE): ")
    print(gpi.apr_perf)
    print("=========================================")
    print("       post performance(RMSE): ")
    print(gpi.post_perf)
    print("=========================================")
print("Training is finished! Shutdown the cluster to release resources.")

Epoch 1/5
39/39 - 1s - loss: 1.4965 - mae: 1.0097 - acc: 0.2862 - val_loss: 1.3360 - val_mae: 0.9692 - val_acc: 0.2422 - 670ms/epoch - 17ms/step
Epoch 2/5
39/39 - 0s - loss: 1.1287 - mae: 0.8863 - acc: 0.2416 - val_loss: 1.0883 - val_mae: 0.8765 - val_acc: 0.2422 - 60ms/epoch - 2ms/step
Epoch 3/5
39/39 - 0s - loss: 0.9632 - mae: 0.8147 - acc: 0.2805 - val_loss: 0.9537 - val_mae: 0.8152 - val_acc: 0.3011 - 56ms/epoch - 1ms/step
Epoch 4/5
39/39 - 0s - loss: 0.8553 - mae: 0.7605 - acc: 0.3604 - val_loss: 0.8517 - val_mae: 0.7616 - val_acc: 0.4141 - 57ms/epoch - 1ms/step
Epoch 5/5
39/39 - 0s - loss: 0.7693 - mae: 0.7120 - acc: 0.5139 - val_loss: 0.7689 - val_mae: 0.7141 - val_acc: 0.5794 - 54ms/epoch - 1ms/step
Epoch 1/5
9/9 - 1s - loss: 0.7398 - mae: 0.6895 - acc: 0.5659 - val_loss: 0.6410 - val_mae: 0.6399 - val_acc: 0.7103 - 1s/epoch - 116ms/step
Epoch 2/5
9/9 - 0s - loss: 0.5704 - mae: 0.5849 - acc: 0.6908 - val_loss: 0.5873 - val_mae: 0.5818 - val_acc: 0.7201 - 40ms/epoch - 4ms/step
E

INFO:tensorflow:Assets written to: ./dnn_examples/results/gpi1/all_years/optimized_model_2014/assets


INFO:tensorflow:Assets written to: /tmp/tmp_model12474447395350038312/assets


INFO:tensorflow:Assets written to: /tmp/tmp_model12474447395350038312/assets


16/16 [==============================] - 0s 821us/step
INFO:tensorflow:Assets written to: /tmp/tmp_model17593858050894971300/assets


...

INFO:tensorflow:Assets written to: /tmp/tmp_model13767359795051392114/assets


52/52 [==============================] - 0s 800us/step
INFO:tensorflow:Assets written to: ./dnn_examples/results/gpi5/best_optimized_model_2015/assets


INFO:tensorflow:Assets written to: ./dnn_examples/results/gpi5/best_optimized_model_2015/assets


       GPI 5 done
       aprior performance(RMSE): 
[[0.24258]
 [0.09105]
 [0.27377]]
       post performance(RMSE): 
[[0.06875]
 [0.13189]
 [0.30721]]
Training is finished! Shutdown the cluster to release resources.


2023-12-05 14:52:28,929 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
